In [2]:
import sys
from PyQt5 import QtWidgets, QtCore, QtGui
from PyQt5.QtWidgets import QSystemTrayIcon, QApplication, QLabel, QMainWindow, QPushButton
from PyQt5.QtGui import QIcon
from PyQt5.QtCore import Qt
import tkinter as tk
import PIL.Image
import numpy as np
import cv2
import io
import json
import ctypes
from io import BytesIO
import win32clipboard
import glob
import os
from matplotlib import pyplot as plt
from IPython.display import clear_output, Image, display
from google.cloud.vision import types
from google.cloud import vision
import mss
import pyperclip
import mss.tools
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/poorv/Downloads/ocr_scripts/keys/direct-outlook-270501-c05c2d97d1c6.json"

In [3]:
def send_text_clipboard(text):
    pyperclip.copy(text)
    spam = pyperclip.paste()
    
def dtlh(path, lh):
    """Detects text in the file."""
    client = vision.ImageAnnotatorClient()
    if type(path) == str:
        with io.open(path, 'rb') as image_file:
            content = image_file.read()
            image = vision.types.Image(content=content)
    else:
        image = types.Image(content=cv2.imencode('.jpg', path)[1].tostring())
        
    response = client.text_detection(
    image=image,
    image_context={"language_hints": lh},
    )
    texts = response.text_annotations
    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
    return texts[0].description

In [4]:
class SystemTrayIcon(QtWidgets.QSystemTrayIcon):

    def __init__(self, icon, parent=None):
        QtWidgets.QSystemTrayIcon.__init__(self, icon, parent)
        menu = QtWidgets.QMenu(parent)
        exitAction = menu.addAction("Exit", self.exit)
        self.setContextMenu(menu)
        self.show()

    def exit(self):
        print("icon exit")
        self.hide()
        QtCore.QCoreApplication.exit()

In [5]:
class ocrWidget(QtWidgets.QWidget):
#     clicked = QtCore.pyqtSignal(object)
    
    def __init__(self, parent = None):
        super().__init__()
        root = tk.Tk()
        screen_width = root.winfo_screenwidth()
        screen_height = root.winfo_screenheight()
        self.setGeometry(0, 0, screen_width, screen_height)
        self.setWindowTitle(' ')
        self.begin = QtCore.QPoint()
        self.end = QtCore.QPoint()
        self.setWindowOpacity(0.3)
        QtWidgets.QApplication.setOverrideCursor(
            QtGui.QCursor(QtCore.Qt.CrossCursor)
        )
        self.setWindowFlags(QtCore.Qt.FramelessWindowHint)
        QtWidgets.QShortcut(
            QtGui.QKeySequence("Escape"), self, activated=self.on_Escape
        )
#         self.mouseReleaseEvent.connect(self.parent.show)
#         image = r'icon.jpg'
#         self.tray_icon = SystemTrayIcon(QtGui.QIcon(image), self)
#         print(type(self))
#         print('Capture the screen...')
#         self.show()
#         testSignal = QtCore.pyqtSignal(object)
#     self.pushButton.clicked.connect(self.parent)


    def paintEvent(self, event):
        qp = QtGui.QPainter(self)
        qp.setPen(QtGui.QPen(QtGui.QColor('black'), 3))
        qp.setBrush(QtGui.QColor(128, 128, 255, 128))
        qp.drawRect(QtCore.QRect(self.begin, self.end))

    def mousePressEvent(self, event):
        self.begin = event.pos()
        self.end = self.begin
        self.update()
    
    @QtCore.pyqtSlot()
    def on_Escape(self):
        print("main esp exit")
        self.close()

    def mouseMoveEvent(self, event):
        self.end = event.pos()
        self.update()
    
    def changeWindow(self):
        self.hide()
        self.close()
        self.parent.show()    

    def mouseReleaseEvent(self, event):
#         self.hide()
#         self.close()
        previous = self.parent
        self.close()

        x1 = min(self.begin.x(), self.end.x())
        y1 = min(self.begin.y(), self.end.y())
        x2 = max(self.begin.x(), self.end.x())
        y2 = max(self.begin.y(), self.end.y())
        
        with mss.mss() as sct:
            # The screen part to capture
            monitor = {"top": y1, "left":x1, "width": abs(x2 - x1) , "height": abs(y2 - y1)}
            
            output = "sct-{top}x{left}_{width}x{height}.png".format(**monitor)

            sct_img = sct.grab(monitor)
            img = PIL.Image.frombytes("RGB", sct_img.size, sct_img.bgra, "raw", "BGRX")

        img = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2RGB)
        im_gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        ret,thresh = cv2.threshold(im_gray,127,255,cv2.THRESH_BINARY_INV)
        ocr = dtlh(thresh, ["hi"])
        send_text_clipboard(ocr)
        print(ocr)
#         self.clicked.emit()
#         previous.show()
        
#         cv2.imshow('Captured Image', img)
#         cv2.waitKey(0)
#         print("image exit")
#         cv2.destroyAllWindows()
#         self.changeWindow()

In [6]:
app = QtWidgets.QApplication(sys.argv)
window = ocrWidget()
window.show()
app.aboutToQuit.connect(app.deleteLater)
sys.exit(app.exec_())

बारिश होने लगी।
बच्चे तालियाँ बजाने लगे ।
माँ मेरा नाम चिल्लाने लगी।
हमें प्यास लगने लगी ।
मुझे महसूस होने लगा कि यह पाट
तो कभी खत्म नहीं होगा।



SystemExit: 0

In [ ]:
"""what do we want.
1. when we start the program, icon appears in sys tray
click sys tray, and gray screen appears to capture 
close the OCR widget (but not the main sys tray, when done)
click the sys tray again to start OCR again 

In [ ]:
class SystemTrayIcon(QtWidgets.QSystemTrayIcon):

    def __init__(self, parent=None, ocr=None):
        QtWidgets.QSystemTrayIcon.__init__(self, parent)
        self.setIcon(QIcon("icon.jpg"))
        menu = QtWidgets.QMenu(parent)
        exitAction = menu.addAction("Exit")
        self.setVisible(True)  
        self.setContextMenu(menu)
        menu.triggered.connect(self.exit)
        self.activated.connect(self.reInit)
        self.ocr = ocr

    def reInit(self, reason):
        if reason == QtWidgets.QSystemTrayIcon.Trigger:
            print("clicked")
            
            self.ocr = ocrWidget(self)
            self.ocr.clicked.connect(self.ri)
            self.ocr.show()
#             if not self.ocr:
#                 self.ocr = ocrWidget(self.parent)
#             self.ocr.show()
            
#             if not self.ocr or isVisibleWidget(self.ocr):
#                 
#                 self.ocr.show()
#             else:
#                 self.ocr.show()
    def ri(self):
        self.ocr = ocrWidget(self)
                
    def exit(self):
        print("exited icon")
        self.hide()
        QtCore.QCoreApplication.exit()
    
    def isVisibleWidget(widget):
        if not widget.visibleRegion().isEmpty():
            return True
        return False

def main():
    app = QtWidgets.QApplication(sys.argv)
#     app.setQuitOnLastWindowClosed(False) 
    main = QtWidgets.QWidget()
#     scanner = ocrWidget()
    trayIcon = SystemTrayIcon(main)
    trayIcon.show()
    sys.exit(app.exec_())
    
main()


# if __name__ == '__main__':
#     on='icon.ico'
#     main(on)

In [15]:
dtlh(r"C:\Users\poorv\Downloads\ocr_scripts\demo\pg191_excerpt.png", ["kn"])

"wego\n?weg 3 erata\n? ಎಬಿವು' eravu\nn. [move) pulling (C. (Kitt.)) [Ka.\nD504(a)]\nn. (move] 1 pulling, dragging\n2 spasm, cramp (My. (Kitt.)) [Ka. D504(a)].\n•ಎಬಿವು' eravu\n283) vt. (move] 1 to rub;\nto rub off or out; to smear gently 2 to stroke gently\n[Ka. D505].\n"

In [6]:
class MainWindow(QMainWindow):

    def __init__(self, *args, **kwargs):
        super(MainWindow, self).__init__(*args, **kwargs)

        self.setWindowTitle("My Awesome App")

        label = QLabel("This is a PyQt5 window!")

        # The `Qt` namespace has a lot of attributes to customise
        # widgets. See: http://doc.qt.io/qt-5/qt.html
        label.setAlignment(Qt.AlignCenter)

        # Set the central widget of the Window. Widget will expand
        # to take up all the space in the window by default.
        self.setCentralWidget(label)
        btn = QPushButton('Button', self)
        btn.clicked.connect(self.reInit)
    
    def reInit(self):
        window = ocrWidget()
        window.show()



In [7]:
app = QApplication(sys.argv)

window = MainWindow()
window.show()

app.exec_()

0